In [1]:
import pandas as pd
import numpy as np
import os

In [25]:
# manually rate the clusters of images

ratings_dict = {0:'mediocre',1:'bad',2:'no_response',3:'mediocre',4:'good',5:'mediocre',6:'no_response',7:'good',
                8:'mediocre',9:'mediocre',10:'good',11:'good',12:'bad',13:'good',14:'no_response',
                15:'mediocre',16:'bad',17:'mediocre',18:'bad',19:'good',20:'mediocre',
                21:'mediocre',22:'no_response',23:'mediocre',24:'bad',25:'bad',26:'mediocre',27:'good',
                28:'no_response',29:'no_response',30:'mediocre',31:'no_response',32:'no_response',33:'good',
                34:'mediocre',35:'no_response',36:'good',37:'mediocre',38:'mediocre',39:'bad'}

In [31]:
# change into a dataframe

labels = pd.DataFrame.from_dict(ratings_dict, orient='index')
labels.reset_index(inplace=True)
labels.columns = ['Cluster','VGG19 Label']
labels

Cluster  VGG19 Label
0         0     mediocre
1         1          bad
2         2  no_response
3         3     mediocre
4         4         good
5         5     mediocre
6         6  no_response
7         7         good
8         8     mediocre
9         9     mediocre
10       10         good
11       11         good
12       12          bad
13       13         good
14       14  no_response
15       15     mediocre
16       16          bad
17       17     mediocre
18       18          bad
19       19         good
20       20     mediocre
21       21     mediocre
22       22  no_response
23       23     mediocre
24       24          bad
25       25          bad
26       26     mediocre
27       27         good
28       28  no_response
29       29  no_response
30       30     mediocre
31       31  no_response
32       32  no_response
33       33         good
34       34     mediocre
35       35  no_response
36       36         good
37       37     mediocre
38       38     mediocre
39       39          bad

In [3]:
# read in the csv file mapping the images to each cluster

clusters = pd.read_csv('../../Final/model/VGG19_kmeans40_ratings.csv')
clusters

image  cluster
0             KP-4_1088_490_Glo_400_4_24.png       35
1          NCI-H650_1034_517_Glo_400_4_3.png       35
2      SK-MEL-2_1266_714_b_17_a_1600_3_0.png       35
3      SK-MEL-2_1266_714_f_17_a_1600_3_0.png       35
4            SUIT-2_1034_495_Glo_325_4_9.png       35
...                                      ...      ...
62154       WIL2-NS_1083_364_Glo_200_4_3.png       38
62155          A549_1053_284_Glo_100_4_6.png       38
62156         PC-14_1003_516_Glo_250_4_6.png       38
62157         SW620_1058_284_Glo_350_4_9.png       38
62158       SNU-C2B_1401_217_Glo_500_4_3.png       38

[62159 rows x 2 columns]

In [21]:
# read in the csv file with the raw data and curve metrics

triplicate_batch_images = pd.read_csv('../../Final/data/triplicate_batch_images.csv')
triplicate_batch_images

cell  drug  min_conc  max_conc  min_norm_intensity  max_norm_intensity  \
0      8570  1034  0.007812       2.0            0.074097            0.664380   
1      8570  1034  0.007812       2.0            0.087338            0.672330   
2      8570  1034  0.007812       2.0            0.068806            0.717342   
3      8570  1034  0.007812       2.0            0.071687            0.569093   
4      8570  1034  0.007812       2.0            0.058225            0.714683   
...     ...   ...       ...       ...                 ...                 ...   
63231  6412  1192  0.009766      10.0            0.307581            1.335618   
63232  6413  1192  0.009766      10.0            0.671477            1.099878   
63233  6290  1192  0.009766      10.0            0.640827            1.023175   
63234  6310  1192  0.009766      10.0            0.572646            1.129622   
63235  6411  1192  0.009766      10.0            0.816443            1.157349   

        mean_sd  convergence  upper_limit         IC50     slope  lower_limit  \
0      0.044709         True     0.901533     0.010218 -2.155016     0.152275   
1      0.040231         True     1.610422     0.005150 -1.883030     0.167002   
2      0.050703         True    14.883637     0.000010 -0.010746   -12.938230   
3      0.038261         True     0.744103     0.011272 -1.882265     0.129222   
4      0.055069         True     0.654130     0.026046 -6.163449     0.129686   
...         ...          ...          ...          ...       ...          ...   
63231  0.078498         True     1.165328     6.381202 -1.442833    -0.115036   
63232  0.064007         True     1.042716    22.063667 -0.497382     0.277841   
63233  0.069889         True     0.911329     0.957016 -9.194456     0.793217   
63234  0.119326         True     0.928182  1782.928576 -0.634245    -6.424923   
63235  0.063773         True     4.173590     0.005404 -5.491619     0.970245   

                                               file_path  \
0      /Users/akfay/Documents/Capstone/triplicate_bat...   
1      /Users/akfay/Documents/Capstone/triplicate_bat...   
2      /Users/akfay/Documents/Capstone/triplicate_bat...   
3      /Users/akfay/Documents/Capstone/triplicate_bat...   
4      /Users/akfay/Documents/Capstone/triplicate_bat...   
...                                                  ...   
63231  /Users/akfay/Documents/Capstone/triplicate_bat...   
63232  /Users/akfay/Documents/Capstone/triplicate_bat...   
63233  /Users/akfay/Documents/Capstone/triplicate_bat...   
63234  /Users/akfay/Documents/Capstone/triplicate_bat...   
63235  /Users/akfay/Documents/Capstone/triplicate_bat...   

                                image_name  
0           PC-14_1034_419_Glo_250_4_3.png  
1           PC-14_1034_419_Glo_250_4_6.png  
2           PC-14_1034_419_Glo_250_4_9.png  
3          PC-14_1034_419_Glo_250_4_12.png  
4          PC-14_1034_419_Glo_250_4_15.png  
...                                    ...  
63231    HCC2218_1192_347_Glo_2000_4_3.png  
63232       AU565_1192_347_Glo_500_4_3.png  
63233    HCC1187_1192_347_Glo_3000_4_3.png  
63234  MDA-MB-231_1192_347_Glo_100_4_3.png  
63235  MDA-MB-436_1192_347_Glo_400_4_3.png  

[63236 rows x 14 columns]

In [34]:
# combine dataframes to map the images to labels

image_label = clusters.merge(labels, left_on='cluster', right_on='Cluster', how='outer')
image_label.drop('Cluster', axis=1, inplace=True)
image_label.rename(columns={'VGG19 Label':'label'}, inplace=True)
image_label

image  cluster        label
0             KP-4_1088_490_Glo_400_4_24.png       35  no_response
1          NCI-H650_1034_517_Glo_400_4_3.png       35  no_response
2      SK-MEL-2_1266_714_b_17_a_1600_3_0.png       35  no_response
3      SK-MEL-2_1266_714_f_17_a_1600_3_0.png       35  no_response
4            SUIT-2_1034_495_Glo_325_4_9.png       35  no_response
...                                      ...      ...          ...
62154       WIL2-NS_1083_364_Glo_200_4_3.png       38     mediocre
62155          A549_1053_284_Glo_100_4_6.png       38     mediocre
62156         PC-14_1003_516_Glo_250_4_6.png       38     mediocre
62157         SW620_1058_284_Glo_350_4_9.png       38     mediocre
62158       SNU-C2B_1401_217_Glo_500_4_3.png       38     mediocre

[62159 rows x 3 columns]

In [35]:
# combine all into one dataframe

triplicate_batch_images_labels = triplicate_batch_images.merge(image_label, left_on='image_name', right_on='image', how='outer')
triplicate_batch_images_labels.drop('image', axis=1, inplace=True)
triplicate_batch_images_labels

cell  drug  min_conc  max_conc  min_norm_intensity  max_norm_intensity  \
0      8570  1034  0.007812       2.0            0.074097            0.664380   
1      8570  1034  0.007812       2.0            0.087338            0.672330   
2      8570  1034  0.007812       2.0            0.068806            0.717342   
3      8570  1034  0.007812       2.0            0.071687            0.569093   
4      8570  1034  0.007812       2.0            0.058225            0.714683   
...     ...   ...       ...       ...                 ...                 ...   
63231  6412  1192  0.009766      10.0            0.307581            1.335618   
63232  6413  1192  0.009766      10.0            0.671477            1.099878   
63233  6290  1192  0.009766      10.0            0.640827            1.023175   
63234  6310  1192  0.009766      10.0            0.572646            1.129622   
63235  6411  1192  0.009766      10.0            0.816443            1.157349   

        mean_sd  convergence  upper_limit         IC50     slope  lower_limit  \
0      0.044709         True     0.901533     0.010218 -2.155016     0.152275   
1      0.040231         True     1.610422     0.005150 -1.883030     0.167002   
2      0.050703         True    14.883637     0.000010 -0.010746   -12.938230   
3      0.038261         True     0.744103     0.011272 -1.882265     0.129222   
4      0.055069         True     0.654130     0.026046 -6.163449     0.129686   
...         ...          ...          ...          ...       ...          ...   
63231  0.078498         True     1.165328     6.381202 -1.442833    -0.115036   
63232  0.064007         True     1.042716    22.063667 -0.497382     0.277841   
63233  0.069889         True     0.911329     0.957016 -9.194456     0.793217   
63234  0.119326         True     0.928182  1782.928576 -0.634245    -6.424923   
63235  0.063773         True     4.173590     0.005404 -5.491619     0.970245   

                                               file_path  \
0      /Users/akfay/Documents/Capstone/triplicate_bat...   
1      /Users/akfay/Documents/Capstone/triplicate_bat...   
2      /Users/akfay/Documents/Capstone/triplicate_bat...   
3      /Users/akfay/Documents/Capstone/triplicate_bat...   
4      /Users/akfay/Documents/Capstone/triplicate_bat...   
...                                                  ...   
63231  /Users/akfay/Documents/Capstone/triplicate_bat...   
63232  /Users/akfay/Documents/Capstone/triplicate_bat...   
63233  /Users/akfay/Documents/Capstone/triplicate_bat...   
63234  /Users/akfay/Documents/Capstone/triplicate_bat...   
63235  /Users/akfay/Documents/Capstone/triplicate_bat...   

                                image_name  cluster        label  
0           PC-14_1034_419_Glo_250_4_3.png       12          bad  
1           PC-14_1034_419_Glo_250_4_6.png       12          bad  
2           PC-14_1034_419_Glo_250_4_9.png       29  no_response  
3          PC-14_1034_419_Glo_250_4_12.png       12          bad  
4          PC-14_1034_419_Glo_250_4_15.png       19         good  
...                                    ...      ...          ...  
63231    HCC2218_1192_347_Glo_2000_4_3.png       38     mediocre  
63232       AU565_1192_347_Glo_500_4_3.png       37     mediocre  
63233    HCC1187_1192_347_Glo_3000_4_3.png        1          bad  
63234  MDA-MB-231_1192_347_Glo_100_4_3.png        6  no_response  
63235  MDA-MB-436_1192_347_Glo_400_4_3.png        1          bad  

[63236 rows x 16 columns]

In [24]:
# save locally

if os.path.exists('../analysis') == False:
    os.makedirs('../analysis')
    
triplicate_batch_images_labels.to_csv('../analysis/triplicate_batch_images_labels_updated.csv')